In [ ]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# dataset_directory = "../Citeseer"
citeseer_dataset = Planetoid(root='', name='Citeseer')
data = citeseer_dataset[0].to(device)
print(data)

In [1]:
trials = 5

In [ ]:
edges_to_add = None

In [ ]:
model = GCN(data.x.shape[1], citeseer_dataset.num_classes, [16]).to(device)

In [ ]:
model_save_path = "../models/citeseer_gcn_model.pth"
list_save_path = "../attacks/citeseer_gcn_edges_full_run.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [ ]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [ ]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

In [ ]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.05)

In [ ]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_mcmc_poisoning(data, train, model, 0.0, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

# regular, secondary attack, not reversed

In [ ]:
splits = [0.0, 0.5]
# splits = [0.0, 0.5, 0.7, 0.9]
# splits = [0.9]
split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_mcmc_poisoning(data, train, model, s, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_acc = dict(split_dic_acc)
split_dic_loss = dict(split_dic_loss)
itrs_one = dict(itrs_one)

In [ ]:
record1 = Recording(split_dic_loss, split_dic_acc, itrs_one, Model.GCN, Dataset.CITESEER, AcceptFn.CONSTANT, SelectFn.BINARY, Reverse.ATTACKED)
record1.save(f'data_collection/{"poisoning"}_{"mcmc"}_{"gcn"}_{"citeseer"}_{"constant"}_{"binary"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_acc, ptb_rate, "MCMC", "binary", "constant", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_loss, ptb_rate, "MCMC", "binary", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [ ]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_mcmc_poisoning(data, train, model, s, edges_to_add, binary_fn, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_decreasing_acc = dict(split_dic_decreasing_acc)
split_dic_decreasing_loss = dict(split_dic_decreasing_loss)
itrs_two = dict(itrs_two)

In [ ]:
record1 = Recording(split_dic_decreasing_loss, split_dic_decreasing_acc, itrs_two, Model.GCN, Dataset.CITESEER, AcceptFn.INCREASING, SelectFn.BINARY, Reverse.ATTACKED)
record1.save(f'data_collection/{"poisoning"}_{"mcmc"}_{"gcn"}_{"citeseer"}_{"increasing"}_{"binary"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_acc, ptb_rate, "MCMC", "binary", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_loss, ptb_rate, "MCMC", "binary", "increasing", "loss", "regular")

In [ ]:
split_dic_decreasing_constant_acc = defaultdict(lambda: defaultdict(list))
split_dic_decreasing_constant_loss = defaultdict(lambda: defaultdict(list))
itrs_three = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_constant_acc[s][j], split_dic_decreasing_constant_loss[s][j], itrs_three[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_constant_acc[s][j], split_dic_decreasing_constant_loss[s][j], itrs_three[s][j] = two_phase_attack_mcmc_poisoning(data, train, model, s, edges_to_add, decreasing_fn, constant_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_decreasing_constant_acc = dict(split_dic_decreasing_constant_acc)
split_dic_decreasing_constant_loss = dict(split_dic_decreasing_constant_loss)
itrs_three = dict(itrs_three)

In [ ]:
record1 = Recording(split_dic_decreasing_constant_loss, split_dic_decreasing_constant_acc, itrs_three, Model.GCN, Dataset.CITESEER, AcceptFn.CONSTANT, SelectFn.DECAYING, Reverse.ATTACKED)
record1.save(f'data_collection/{"poisoning"}_{"mcmc"}_{"gcn"}_{"citeseer"}_{"constant"}_{"decaying"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_constant_acc, ptb_rate, "MCMC", "decaying", "constant", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_constant_loss, ptb_rate, "MCMC", "decaying", "constant", "loss", "regular")

In [ ]:
split_dic_decreasing_gradual_acc = defaultdict(lambda: defaultdict(list))
split_dic_decreasing_gradual_loss = defaultdict(lambda: defaultdict(list))
itrs_four = defaultdict(lambda: defaultdict(int))

In [ ]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_gradual_acc[s][j], split_dic_decreasing_gradual_loss[s][j], itrs_four[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_gradual_acc[s][j], split_dic_decreasing_gradual_loss[s][j], itrs_four[s][j] = two_phase_attack_mcmc_poisoning(data, train, model, s, edges_to_add, decreasing_fn, increasing_fn, device, is_reversed=False, verbose=True)

In [ ]:
split_dic_decreasing_gradual_acc = dict(split_dic_decreasing_gradual_acc)
split_dic_decreasing_gradual_loss = dict(split_dic_decreasing_gradual_loss)
itrs_four = dict(itrs_four)

In [ ]:
record1 = Recording(split_dic_decreasing_gradual_loss, split_dic_decreasing_gradual_acc, itrs_four, Model.GCN, Dataset.CITESEER, AcceptFn.CONSTANT, SelectFn.DECAYING, Reverse.ATTACKED)
record1.save(f'data_collection/{"poisoning"}_{"mcmc"}_{"gcn"}_{"citeseer"}_{"increasing"}_{"decaying"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_gradual_acc, ptb_rate, "MCMC", "decaying", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_gradual_loss, ptb_rate, "MCMC", "decaying", "increasing", "accuracy", "regular")